In [3]:
import requests, pandas as pd, time
from pathlib import Path
import datetime as dt
BASE_URL = "https://goods.musinsa.com/api2/review/v1/view/list"
import re
def _num(v):
    if v is None: return None
    m = re.search(r'(\d+(?:\.\d+)?)', str(v))  # '169', '169cm' 모두 처리
    return float(m.group(1)) if m else None
def pick(*vals):
    """첫 번째로 유효한(빈문자/None 아님) 값을 반환"""
    for v in vals:
        if v is None:
            continue
        if isinstance(v, str) and v.strip() == "":
            continue
        return v
    return None

# 2) 리뷰 수집 함수 (goodsOption/성별 보강 포함)
import requests, time
def fetch_reviews_for_product(goods_no: str,
                              page_size: int = 50,   # 10→50/100 으로 늘려도 됨
                              max_pages: int = 200,
                              sort: str = "up_cnt_desc",
                              has_photo: bool = False,
                              is_experience: bool = False,
                              my_filter: bool = False):
    """무신사 리뷰 API에서 필요한 필드만 수집"""
    rows = []
    sess = requests.Session()
    sess.headers.update({
        "User-Agent": "Mozilla/5.0",
        "Accept": "application/json, text/plain, */*",
        "Referer": f"https://www.musinsa.com/review/goods/{goods_no}",
    })
    for page in range(0, max_pages):
        params = {
            "page": page,
            "pageSize": page_size,
            "goodsNo": str(goods_no),
            "sort": sort,
            "selectedSimilarNo": str(goods_no),
            "myFilter": str(my_filter).lower(),
            "hasPhoto": str(has_photo).lower(),
            "isExperience": str(is_experience).lower(),
        }
        r = sess.get(BASE_URL, params=params, timeout=15)
        r.raise_for_status()
        j = r.json()
        items = (((j or {}).get("data") or {}).get("list")) or []
        if not items:
            break
        for it in items:
            goods = it.get("goods") or {}
            user  = it.get("userProfileInfo") or {}
            h_cm = _num(user.get("userHeight") or user.get("height") or it.get("userHeight"))
            w_kg = _num(user.get("userWeight") or user.get("weight") or it.get("userWeight"))
            rows.append({
                "product_id": str(goods_no),
                "review_no": it.get("no") or it.get("id"),
                "content": it.get("content"),
                "brandName": goods.get("brandName") or goods.get("brand"),
                "goodsName": goods.get("goodsName"),
                "goodsOption": pick(
                it.get("goodsOption"),
                goods.get("goodsOption"),
                it.get("option"),
                it.get("sizeOption"),
                it.get("optionName"),
                it.get("orderOptionNm"),
            ),
                "goodsSex": pick(
                it.get("goodsSex"),
                goods.get("goodsSex"),
                goods.get("goodsSexClassification"),
            ),
                "reviewSex": user.get("reviewSex") or user.get("sex"),
                "userHeight_cm": h_cm,
                "userWeight_kg": w_kg,
                "likeCount": it.get("likeCount") or it.get("helpfulCount"),
            })
        # 마지막 페이지 추정: 반환 수가 pageSize보다 작으면 종료
        if len(items) < page_size:
            break
        time.sleep(0.2)
    return rows
# --- 사용 예: 질문 주신 상품(4793690) ---
one = fetch_reviews_for_product("4793690", page_size=50, sort="up_cnt_desc")
df_reviews = pd.DataFrame(one)
print("리뷰 건수:", len(df_reviews))
print(df_reviews.head())

df_reviews.to_csv("adidas_reviews.csv", index=False, encoding="utf-8-sig")
print("CSV 저장 완료!")


리뷰 건수: 768
  product_id  review_no                                            content  \
0    4793690   73489154  ‼️아디다스 정품 사세요‼️\n이거 >>>>>무신사 에디션<<<<<인데\n안감이랑 ...   
1    4793690   71682165  아이가 입고 싶다고 해서 구매했어요 니트재질이네요 \n신축성도 있고 기장은 긴편 슬...   
2    4793690   71625362  배송 하루만에 와서 엄청 좋았어요! 오프 화이트는 생각보다 옷과 매치하기 힘들 것 ...   
3    4793690   71614593                        배송도 생각보다 빨랐고 구매 한 제품도 만족합니다   
4    4793690   74931048  와플 BB 트랙탑 - 블랙 / JV9264S / 한 달 사용 리뷰\n\n한 달 정도...   

  brandName                goodsName goodsOption  goodsSex reviewSex  \
0      아디다스  와플 BB 트랙탑 - 블랙 / JV9264           S         0        여성   
1      아디다스  와플 BB 트랙탑 - 블랙 / JV9264           M         0        여성   
2      아디다스  와플 BB 트랙탑 - 블랙 / JV9264           L         0        여성   
3      아디다스  와플 BB 트랙탑 - 블랙 / JV9264           L         0        남성   
4      아디다스  와플 BB 트랙탑 - 블랙 / JV9264           S         0        남성   

   userHeight_cm  userWeight_kg  likeCount  
0          161.0           47.0      140.0

In [ ]:
import streamlit as st
import requests
import pandas as pd
import time, re
from datetime import datetime

BASE_URL = "https://goods.musinsa.com/api2/review/v1/view/list"

def pick(*vals):
    for v in vals:
        if v is None: 
            continue
        if isinstance(v, str) and v.strip() == "":
            continue
        return v
    return None

def fetch_reviews_for_product(goods_no: str,
                              page_size: int = 50,
                              max_pages: int = 200,
                              sort: str = "up_cnt_desc"):
    """무신사 리뷰 API에서 필요한 필드 수집"""
    rows = []
    sess = requests.Session()
    sess.headers.update({
        "User-Agent": "Mozilla/5.0",
        "Accept": "application/json, text/plain, */*",
        "Referer": f"https://www.musinsa.com/review/goods/{goods_no}",
    })

    for page in range(0, max_pages):
        params = {
            "page": page,
            "pageSize": page_size,
            "goodsNo": str(goods_no),
            "sort": sort,
            "selectedSimilarNo": str(goods_no),
        }
        r = sess.get(BASE_URL, params=params, timeout=15)
        if r.status_code != 200:
            break

        j = r.json()
        items = (((j or {}).get("data") or {}).get("list")) or []
        if not items:
            break

        for it in items:
            goods = it.get("goods") or {}
            user  = it.get("userProfileInfo") or {}

            raw_date = it.get("createDate")
            if raw_date:
                dt_obj = datetime.fromisoformat(raw_date.replace("Z", "+00:00"))
                new_date = dt_obj.strftime("%Y-%m-%d")
            else:
                new_date = None

            rows.append({
                "product_id": str(goods_no),
                "review_no": it.get("no"),
                "brandName": goods.get("brandName"),
                "goodsName": goods.get("goodsName"),
                "price": goods.get("price") or None,   # ⚠️ 없으면 None
                "thumbnail": "https://image.msscdn.net" + goods["goodsImageFile"] if goods.get("goodsImageFile") else None,  # ✅ 전체 URL
                "goodsLinkUrl": f"https://www.musinsa.com/products/{goods_no}",
                "userNickName": user.get("userNickName"), 
                "reviewSex": user.get("reviewSex") or user.get("sex"),# ✅ 리뷰 성별 우선, 없으면 상품 성별
                "grade": it.get("grade"),
                "content": it.get("content"),
                "createDate": new_date,
            })

        if len(items) < page_size:
            break
        time.sleep(0.2)
    return rows


# 예시 실행
if __name__ == "__main__":
    goods_no = "4397727"  # 아디다스 예시
    reviews = fetch_reviews_for_product(goods_no)
    df = pd.DataFrame(reviews)
    print(df.head())

    # CSV 저장
    df.to_csv("reviews_with_product_info.csv", index=False, encoding="utf-8-sig")
    print("CSV 저장 완료!")


리뷰 수: 768
  product_id  review_no                                            content  \
0    4793690   73489154  ‼️아디다스 정품 사세요‼️\n이거 >>>>>무신사 에디션<<<<<인데\n안감이랑 ...   
1    4793690   71682165  아이가 입고 싶다고 해서 구매했어요 니트재질이네요 \n신축성도 있고 기장은 긴편 슬...   
2    4793690   71625362  배송 하루만에 와서 엄청 좋았어요! 오프 화이트는 생각보다 옷과 매치하기 힘들 것 ...   
3    4793690   71614593                        배송도 생각보다 빨랐고 구매 한 제품도 만족합니다   
4    4793690   74931048  와플 BB 트랙탑 - 블랙 / JV9264S / 한 달 사용 리뷰\n\n한 달 정도...   

  brandName                goodsName price  \
0      아디다스  와플 BB 트랙탑 - 블랙 / JV9264  None   
1      아디다스  와플 BB 트랙탑 - 블랙 / JV9264  None   
2      아디다스  와플 BB 트랙탑 - 블랙 / JV9264  None   
3      아디다스  와플 BB 트랙탑 - 블랙 / JV9264  None   
4      아디다스  와플 BB 트랙탑 - 블랙 / JV9264  None   

                                           thumbnail goodsLinkUrl goodsOption  \
0  https://image.msscdn.net/images/goods_img/2025...         None           S   
1  https://image.msscdn.net/images/goods_img/2025...         None           M   
2

In [ ]:
import requests, pandas as pd, time, re
from datetime import datetime

REVIEW_BASE_URL = "https://goods.musinsa.com/api2/review/v1/view/list"
PRODUCT_BASE_URL = "https://api.musinsa.com/api2/dp/v1/goods"


def pick(*vals):
    """첫 번째 유효값 반환"""
    for v in vals:
        if v is None: continue
        if isinstance(v, str) and v.strip() == "": continue
        return v
    return None

def fetch_product_info(goods_no: str):
    """상품 API에서 가격, 썸네일, 브랜드, 링크 가져오기"""
    url = f"{PRODUCT_BASE_URL}?goodsNoList={goods_no}&salesStat=SALE,SOLD_OUT"
    r = requests.get(url, timeout=15)
    if r.status_code != 200:
        return {}
    data = (r.json() or {}).get("data") or []
    if not data:
        return {}
    g = data[0]
    return {
        "price": g.get("price", {}).get("price"),
        "thumbnail": g.get("imageUrl"),
        "brandName": g.get("brandName"),
        "goodsName": g.get("goodsName"),
        "goodsLinkUrl": f"https://www.musinsa.com/products/{g.get('goodsNo')}"
    }

def fetch_reviews_for_product(goods_no: str,
                              page_size: int = 50,
                              max_pages: int = 200,
                              sort: str = "up_cnt_desc"):
    rows = []
    sess = requests.Session()
    sess.headers.update({
        "User-Agent": "Mozilla/5.0",
        "Accept": "application/json, text/plain, */*",
        "Referer": f"https://www.musinsa.com/review/goods/{goods_no}",
    })

    # 상품 정보 먼저 가져오기
    product_info = fetch_product_info(goods_no)

    for page in range(max_pages):
        params = {
            "page": page,
            "pageSize": page_size,
            "goodsNo": str(goods_no),
            "sort": sort,
            "selectedSimilarNo": str(goods_no),
        }
        r = sess.get(REVIEW_BASE_URL, params=params, timeout=15)
        if r.status_code != 200:
            break

        j = r.json()
        items = (((j or {}).get("data") or {}).get("list")) or []
        if not items:
            break

        for it in items:
            goods = it.get("goods") or {}
            user  = it.get("userProfileInfo") or {}

            # 날짜 변환
            raw_date = it.get("createDate")
            if raw_date:
                try:
                    dt_obj = datetime.fromisoformat(raw_date.replace("Z", "+00:00"))
                    new_date = dt_obj.strftime("%Y-%m-%d")
                except:
                    new_date = raw_date
            else:
                new_date = None

            rows.append({
                "product_id": str(goods_no),
                "brandName": pick(product_info.get("brandName"), goods.get("brandName")),
                "goodsName": pick(product_info.get("goodsName"), goods.get("goodsName")),
                "thumbnail": product_info.get("thumbnail") or ("https://image.msscdn.net" + goods.get("goodsImageFile","")
                    if goods.get("goodsImageFile") else None
                ),
                "goodsLinkUrl": f"https://www.musinsa.com/products/{goods_no}",

                "review_no": it.get("no"),
                "createDate": new_date, 
                "userNickName": user.get("userNickName"),
                "reviewSex": user.get("reviewSex") or user.get("sex"),
                "grade": it.get("grade"),
                "content": it.get("content")
            })

        if len(items) < page_size:
            break
        time.sleep(0.2)

    return rows

# 실행 예시
if __name__ == "__main__":
    goods_no = "4793690"
    reviews = fetch_reviews_for_product(goods_no, page_size=50)
    df = pd.DataFrame(reviews)
    print("리뷰 수:", len(df))
    print(df.head())
    df.to_csv("reviews_with_product_info.csv", index=False, encoding="utf-8-sig")
    print("CSV 저장 완료")


리뷰 수: 768
  product_id  review_no                                            content  \
0    4793690   73489154  ‼️아디다스 정품 사세요‼️\n이거 >>>>>무신사 에디션<<<<<인데\n안감이랑 ...   
1    4793690   71682165  아이가 입고 싶다고 해서 구매했어요 니트재질이네요 \n신축성도 있고 기장은 긴편 슬...   
2    4793690   71625362  배송 하루만에 와서 엄청 좋았어요! 오프 화이트는 생각보다 옷과 매치하기 힘들 것 ...   
3    4793690   71614593                        배송도 생각보다 빨랐고 구매 한 제품도 만족합니다   
4    4793690   74931048  와플 BB 트랙탑 - 블랙 / JV9264S / 한 달 사용 리뷰\n\n한 달 정도...   

  brandName                goodsName price  \
0      아디다스  와플 BB 트랙탑 - 블랙 / JV9264  None   
1      아디다스  와플 BB 트랙탑 - 블랙 / JV9264  None   
2      아디다스  와플 BB 트랙탑 - 블랙 / JV9264  None   
3      아디다스  와플 BB 트랙탑 - 블랙 / JV9264  None   
4      아디다스  와플 BB 트랙탑 - 블랙 / JV9264  None   

                                           thumbnail goodsLinkUrl  goodsSex  \
0  https://image.msscdn.net/images/goods_img/2025...         None         0   
1  https://image.msscdn.net/images/goods_img/2025...         None         0   
2  http